In [1]:
using Distributions
using Gadfly
using TimerOutputs

In [8]:
include("../src/ssm.jl")
include("../src/dfm.jl");

In [9]:
n = 20

μ = zeros(n)
λ = randn(n)
ϕ = 0.95
σ2v = 1.
ψ = 0.90*ones(n)
σ2w = [0.5*ones(n/2)..., 1.*ones(n/2)...]

dfm = MixedFreqDFM(μ, λ, ϕ, σ2v, ψ, σ2w)
ssm = get_state_space_representation(dfm)

T1 = 300
ΔT = 24     # 2 years
T = T1 + ΔT
x, y_unobs = simulate(ssm, T)

y = fill(NaN, T, n)
y[3:3:T1,1] = y_unobs[3:3:T1,1]
y[1:T1,2:n] = y_unobs[1:T1,2:n]

y

324×20 Array{Float64,2}:
 NaN          4.17827    4.21231  …   -5.33937    6.95579   -1.36753
 NaN          4.24817    3.83587      -4.70642    5.82448   -2.65438
   8.97427    3.65847    4.20116      -4.55248    5.24341   -3.60635
 NaN          4.62858    4.48362      -4.22738    5.48479   -4.66806
 NaN          4.03777    3.42797      -4.43718    5.01936   -3.33842
   6.33844    2.57326    2.79114  …   -3.77558    3.4229    -3.9176 
 NaN          2.26368    3.37736      -4.2432     5.18676   -2.78738
 NaN          3.32125    4.95931      -3.64724    6.87475   -2.92985
   2.20842    3.39572    3.71878      -4.27685    7.66086   -3.16255
 NaN          2.9122     3.90674      -4.22773    7.52143   -3.34204
 NaN          2.83153    2.56071  …   -3.14362    6.98786   -4.05882
   6.22949    2.88426    1.99943      -4.40453    6.78511   -4.43883
 NaN          3.27745    1.4385       -5.70784    6.86947   -4.7023 
   ⋮                              ⋱                                 
 NaN     

In [10]:
function simulation_smoother(y, ssm)
    
    xs = fast_state_smoothing(y, ssm)
    x1, y1 = simulate(ssm, size(y,1))
    y1[isnan.(y)] = NaN
    xs1  = fast_state_smoothing(y1, ssm)
    x_mcmc = x1 - xs1 + xs
    
    return x_mcmc
    
end

simulation_smoother (generic function with 1 method)

In [14]:
n, k = size(ssm.B)

S = 100

x_all = zeros(S, T, k)

to = TimerOutput()

for s=1:S

    @timeit to "simulation smoother" begin

        dfm_mcmc = MixedFreqDFM(μ, λ, ϕ, σ2v, ψ, σ2w)
        ssm_mcmc = get_state_space_representation(dfm_mcmc)
        x1, y1 = simulate(ssm_mcmc, size(y,1))
        y1[isnan.(y)] = NaN
        @timeit to "fast state smoothing" begin
            xs  = fast_state_smoothing(y,  ssm_mcmc)
            xs1 = fast_state_smoothing(Matrix(y1), ssm_mcmc)
        end
        x_mcmc = x1 - xs1 + xs

    end
    
    x_all[s,:,:] = x_mcmc
    
end

In [15]:
show(to)

 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            33.5s / 100%            8.20GiB / 100%     

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 simulation smoother      100    33.5s   100%   335ms   8.20GiB  100%   84.0MiB
   fast state smoot...    100    32.1s  96.1%   321ms   8.08GiB  98.5%  82.7MiB
 ──────────────────────────────────────────────────────────────────────────────

In [ ]:
plot(
    [layer(x=1:T, y=[quantile(x_all[:,t,1],q) for t=1:T], Geom.line) for q=[0.025, 0.975]]...,
    layer(x=1:T, y=x[:,1], Geom.line, Theme(default_color="red")),
)

In [ ]:
for 